### Algorithms for Data Science
#### Linear Regression and kNN

In [38]:
import  pandas as pd
from statistics import mean, variance
from collections import Counter
import math
import numpy as np
from scipy.stats import gmean
from decimal import Decimal

Consider the following data set for questions Q 1 ∼ Q 20 where<br>
x column is the hours that a student studied, <br>
y column is the hours that a student played video games, and <br>
g is the GPA that the student received <br>

x y g  <br>
5 2 4  <br>
6 5 3  <br>
4 2 3  <br>
3 3 3  <br>
4 7 2  <br>
3 6 2  <br>
1 8 1  

In [2]:
#features = [[5,2],[6,5],[4,2],[3,3],[4,7],[3,6],[1,8]]
#y = [4,3,3,3,2,2,1]

In [3]:
# created train/ referance dataset from given values so that I can use the same code for any dataframe
data = {'x': [5, 6, 4, 3, 4, 3, 1], 'y': [2, 5, 2, 3, 7, 6, 8], 'g': [4, 3 , 3, 3, 2, 2, 1]}  

# Create DataFrame  
df = pd.DataFrame(data)  
train_set = df.copy()

print(df) 

   x  y  g
0  5  2  4
1  6  5  3
2  4  2  3
3  3  3  3
4  4  7  2
5  3  6  2
6  1  8  1


In [4]:
# find the min max of the column to normalize the dataset
def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())

In [5]:
# Calculate the Euclidean distance L2 between two vectors
def euclidean_distance(test_row, train_row):
    distance = 0.0
    for i in range(len(train_row)-1):
        distance += (test_row[i] - train_row[i])**2
    return math.sqrt(distance)

In [6]:
#create function to calculate Manhattan distance 
def manhattan_distance(a, b):
    distance = 0.0

    for i in range(len(a)-1):
        distance += abs(a[i] - b[i])
    
    return distance 

In [7]:
# Function distance between two points

def p_root(value, root):
     
    root_value = 1 / float(root)
    return round (Decimal(value) **
             Decimal(root_value), 3)

In [8]:
#create function to calculate minkowski distance 

def minkowski_distance(x, y, p_value):
   
    # pass the p_root function to calculate
    # all the value of vector parallelly
    return (p_root(sum(pow(abs(a-b), p_value)
            for a, b in zip(x, y)), p_value))


In [9]:
def get_geometric_mean(data):
    multiplication = 1
    
    cnt = len(data)

    for ele in data:
        multiplication = (multiplication)*(ele)

    geometric_mean = (multiplication)**(1/cnt)
    
    return geometric_mean

In [10]:
# Function to find arithmetic geometric mean
def get_arithmetic_geometric_mean(lst, tolerance=1e-10):

    a0 = mean(lst) # Get the arithmetic mean
    g0 = gmean(lst) # Get the geomatric mean
    
    an, gn = (a0 + g0) / 2.0, math.sqrt(a0 * g0)
    while abs(an - gn) > tolerance:
        an, gn = (an + gn) / 2.0, math.sqrt(an * gn)
    
    return an

In [11]:
# Function to get distance weighted neighbour
def get_distance_weighted_neighbour(neighbors, p):
    
    # print('neighbors', neighbors)
    wkNNPred = 0
    wNN = [0]*len(neighbors)

    for i in range(len(neighbors)):
        # Get the distance value from list
        distance = neighbors[i][1]
        #print('neighbors[i][0][2]', neighbors[i][0][2])
        
        #Get the weighted distance for each neighbor
        if distance == 0:  #To avoid divide by zero error
            wNN[i] = 0
        else:
            wNN[i] = 1/(distance**p)
            # multiply weighted distance by truth i.e Grade in our case and add them 
            truth = neighbors[i][0][2]
            wkNNPred+= wNN[i]*truth    
    
    # Divide sum of (weighted distance*truth) by sum of weighted distance
    if sum(wNN) > 0:  #To avoid divide by zero error (rare case)
        return wkNNPred/sum(wNN)
    else:
        return 0

    

In [12]:
# Get the most similar neighbors
def get_neighbors(train_set, test_row, k_neighbors, distancemethod, p, p_minkowski):
    
    distances = list()
    neighbors = list()
    
    for train_row in train_set:
        if(distancemethod == 'euclidean'):
            dist = euclidean_distance(test_row, train_row)
            distances.append((train_row, dist))
            
        elif(distancemethod == 'manhattan'):
            
            dist = manhattan_distance(test_row, train_row)
            distances.append((train_row, dist))
            
        elif(distancemethod == 'minkowski'):           
            # p_minkowski value needed
            train_row_new = train_row[:-1]
            dist = minkowski_distance(test_row, train_row_new, p_minkowski)
            
            #Library function does not accept .5 value
            #dist = distance.minkowski(test_row, train_row_new, p)
            distances.append((train_row, dist))
            
        else:
            print('Please provide valid method to calculate distance')

    # we can print the distances just to cross check our answer
    #print('distances', distances)
    
    # sort the distances
    distances.sort(key=lambda tup: tup[1])
    
    # if p>0 i]that means we have to calculated weighted distances  
    if p > 0:
        # using first k distances
        kneighbor = distances[:k_neighbors]

        #call function to calculate weighted distances   
        predicted_val = get_distance_weighted_neighbour(kneighbor, p)
        return predicted_val
    
    else:
        for i in range(k_neighbors):
            neighbors.append(distances[i][0])  
    return neighbors

In [13]:

# Make a prediction with neighbors
def predict_classification(train, test_row, k_neighbors, distancemethod, p, p_minkowski, averagemethod):
    if p <= 0:
        neighbors = get_neighbors(train, test_row, k_neighbors, distancemethod, p, p_minkowski)
        
        #Get all the grades of nearest neighbors
        output_values = [row[-1] for row in neighbors]
    
        # Get the average of k nearest neighbores as per given method
        if averagemethod == 'geometric average':
            prediction = get_geometric_mean(output_values)
            
        elif averagemethod == 'arithmetic-geometric average':
            prediction = get_arithmetic_geometric_mean(output_values)
            
        else:
            prediction = round(mean(output_values), 2)
            
    else:
        prediction = get_neighbors(train, test_row, k_neighbors, distancemethod, p, p_minkowski)
            
    return prediction

In [14]:
# kNN Algorithm
# Parameters 
    # train is the train dataset
    # test is the test dataset
    # k_neighbors = no. of numbers need to find out
    # distancemethod = method for distance calculation
    # p_minkowski for minkowski distance calculations
    # p for weighted distance KNN
    
def get_nearest_neighbors_prediction(train, test, k_neighbors, distancemethod, p = 0, p_minkowski = 0, averagemethod = 'simple average'):
      
    train_list = train.values.tolist()
  
    predicted_output = predict_classification(train_list, test, k_neighbors, distancemethod, p, p_minkowski, averagemethod)

    return round(predicted_output, 2)


#### Q 1. According to the nearest neighbor algorithm where Euclidean L2 distance is used, what is the expected GPA if x = 2 and y = 4?

In [15]:
#Manually calculated distance for x = 2 and y = 4 using Euclidean L2 method <br>  
# We can print distances in function itself, but I dont want to print distances for every function is called so I am adding it here manually

distances = {'x': [5, 6, 4, 3, 4, 3, 1], 'y': [2, 5, 2, 3, 7, 6, 8], 'distance': [3.60, 4.12 , 2.82, 1.41, 3.60, 2.24, 4.12], 'g': [4, 3 , 3, 3, 2, 2, 1]}  

# Create DataFrame  
distance_df = pd.DataFrame(distances)  
distance_df = distance_df.sort_values(by=['distance'])
print(distance_df) 
index = 3
# From the result we can see the min distance is 1.41 which has grade 3 therfore the code should 
# return grade 3 for x = 2 and y = 4 

# using the code to calculate distances
test_set = [2, 4]

predicted_GPA = get_nearest_neighbors_prediction(train_set, test_set, 1, distancemethod ='euclidean')

print('\n\nFor x = 2 and y = 4, index of expected gpa is {0} Expected GPA is: {1}'.format(index, predicted_GPA))


   x  y  distance  g
3  3  3      1.41  3
5  3  6      2.24  2
2  4  2      2.82  3
0  5  2      3.60  4
4  4  7      3.60  2
1  6  5      4.12  3
6  1  8      4.12  1


For x = 2 and y = 4, index of expected gpa is 3 Expected GPA is: 3


#### Q 2. According to the (k = 3)-NN algorithm where Euclidean L2 distance is used, what is the expected GPA if x = 2 and y = 4?

In [16]:
test_set = [2, 4]

predicted_GPA= get_nearest_neighbors_prediction(train_set, test_set, k_neighbors=3, distancemethod ='euclidean')

print('\nFor x = 2 and y = 4, Expected GPA is:' , predicted_GPA)



For x = 2 and y = 4, Expected GPA is: 2.67


####  Q 3. According to the (k = 5)-NN algorithm where Euclidean L2 distance is used, what is the expected GPA if x = 2 and y = 4?

In [17]:
test_set = [2, 4]

predicted_GPA= get_nearest_neighbors_prediction(train_set, test_set, k_neighbors=5, distancemethod ='euclidean')

print('\nFor x = 2 and y = 4, Expected GPA is:' , predicted_GPA)



For x = 2 and y = 4, Expected GPA is: 2.8


#### Q 4. According to the geometric average (k = 3)-NN algorithm where Euclidean L2 distance is used, what is the expected GPA if x = 2 and y = 4?

In [18]:
test_set = [2, 4]

predicted_GPA= get_nearest_neighbors_prediction(train_set, test_set, k_neighbors=3, distancemethod ='euclidean', averagemethod='geometric average')

print('\nFor x = 2 and y = 4, Expected GPA is:' , predicted_GPA)



For x = 2 and y = 4, Expected GPA is: 2.62


#### Q 5. According to the arithmetic-geometric average (k = 3)-NN algorithm where Euclidean L2 distance is used, what is the expected GPA if x = 2 and y = 4?

In [19]:
test_set = [2, 4]

predicted_GPA= get_nearest_neighbors_prediction(train_set, test_set, k_neighbors=3, distancemethod ='euclidean', averagemethod='arithmetic-geometric average')

print('\n For x = 2 and y = 4, Expected GPA is:' , predicted_GPA)



 For x = 2 and y = 4, Expected GPA is: 2.64


#### Q 6. According to the (p = 2) distance weighted (k = 5)-NN algorithm where Euclidean L2 distance is used, what is the expected GPA if x = 2 and y = 4?

In [20]:
test_set = [2, 4]

predicted_GPA= get_nearest_neighbors_prediction(train_set, test_set, k_neighbors=5, distancemethod ='euclidean', p=2)

print('\nFor x = 2 and y = 4, distance weighted Expected GPA is:' , predicted_GPA)



For x = 2 and y = 4, distance weighted Expected GPA is: 2.8


#### Q 7. According to the (p = 3) distance weighted (k = 5)-NN algorithm where Euclidean L2 distance is used, what is the expected GPA if x = 2 and y = 4?

In [21]:
test_set = [2, 4]

predicted_GPA= get_nearest_neighbors_prediction(train_set, test_set, k_neighbors=5, distancemethod ='euclidean', p= 3)

print('\nFor x = 2 and y = 4, distance weighted Expected GPA is:' , predicted_GPA)



For x = 2 and y = 4, distance weighted Expected GPA is: 2.83


#### Q 8. According to the (k = 3)-NN algorithm where Manhattan L1 distance is used, what is the expected GPA if x = 6 and y = 2?


In [22]:
query_set = [6, 2]

predicted_GPA= get_nearest_neighbors_prediction(train_set, query_set, k_neighbors=3, distancemethod ='manhattan')

print('\nFor x = 6 and y = 2, Expected GPA is:' , predicted_GPA)



For x = 6 and y = 2, Expected GPA is: 3.33


#### Q 9. According to the (k = 5)-NN algorithm where Manhattan L1 distance is used, what is the expected GPA if x = 6 and y = 2?

In [23]:
query_set = [6, 2]

predicted_GPA= get_nearest_neighbors_prediction(train_set, query_set, k_neighbors=5, distancemethod ='manhattan')

print('\nFor x = 6 and y = 2, Expected GPA is:' , predicted_GPA)



For x = 6 and y = 2, Expected GPA is: 3


#### Q 10. According to the (p = 2) distance weighted (k = 5)-NN algorithm where Manhattan L1 distance is used, what is the expected GPA if x = 6 and y = 2?

In [24]:
query_set = [6, 2]

predicted_GPA= get_nearest_neighbors_prediction(train_set, query_set, k_neighbors=5, distancemethod ='manhattan', p=2)

print('\nFor x = 6 and y = 2, distance weighted Expected GPA is:' , predicted_GPA)



For x = 6 and y = 2, distance weighted Expected GPA is: 3.47


#### Q 11. What does the (p = 3) distance weighted (k = 5)-NN algorithm classifies a query instance (6, 2) if Manhattan L1 distance is used.

In [25]:
query_set = [6, 2]

predicted_GPA= get_nearest_neighbors_prediction(train_set, query_set, k_neighbors=5, distancemethod ='manhattan', p=3)

print('\nFor x = 6 and y = 2, Expected GPA is:' , predicted_GPA)



For x = 6 and y = 2, Expected GPA is: 3.68


#### Q 12. According to the (k = 3)-NN algorithm where Minkowski (p = 3) L3 distance is used, what is the expected GPA  if x = 6 and y = 2?


In [26]:
query_set = [6, 2]

predicted_GPA= get_nearest_neighbors_prediction(train_set, query_set, k_neighbors=3, distancemethod ='minkowski', p_minkowski=3)

print('\nFor x = 6 and y = 2, Expected GPA is:' , predicted_GPA)




For x = 6 and y = 2, Expected GPA is: 3.33


#### Q 13. Normalize the dataset using the simple min-max normalization

In [27]:
# normalize the each colum of dataset expect truth i.e 'g' using simple min max scale

normalized_traindf = train_set.copy()
# adding query to dataframe to normalize it
query_df = {'x': 6, 'y': 2}

normalized_traindf = normalized_traindf.append(query_df, ignore_index = True)
print('\n normalized traindf with query value at 7th position\n')
for col in normalized_traindf.columns:
    if col != 'g':
        normalized_traindf[col] = min_max_scaling(normalized_traindf[col])
print(normalized_traindf)

# delete test set from train set
normalized_traindf = normalized_traindf.drop(labels=7, axis=0)
print('\n')
#print normalized data set
normalized_traindf


 normalized traindf with query value at 7th position

     x         y    g
0  0.8  0.000000  4.0
1  1.0  0.500000  3.0
2  0.6  0.000000  3.0
3  0.4  0.166667  3.0
4  0.6  0.833333  2.0
5  0.4  0.666667  2.0
6  0.0  1.000000  1.0
7  1.0  0.000000  NaN




,x,y,g
0,0.8,0.000000,4.0
1,1.0,0.500000,3.0
2,0.6,0.000000,3.0
3,0.4,0.166667,3.0
4,0.6,0.833333,2.0
5,0.4,0.666667,2.0
6,0.0,1.000000,1.0


#### Q 14. According to the (k = 3)-NN algorithm where Euclidean L2 distance is used on the normalized dataset in Q 13, what is the expected GPA if x = 6 and y = 2?

In [28]:
query_set = [6, 2]
normalized_query_set = [1, 0]

predicted_GPA= get_nearest_neighbors_prediction(train_set, normalized_query_set, k_neighbors=3, distancemethod ='euclidean')

print('\nFor x = 6 and y = 2, using normalized query values - Expected GPA is:' , predicted_GPA)

predicted_GPA1= get_nearest_neighbors_prediction(train_set, query_set, k_neighbors=3, distancemethod ='euclidean')

print('\n\nFor x = 6 and y = 2, using original query values - Expected GPA is:' , predicted_GPA1)

# Getting same expected GPA for both normalized and original values of query



For x = 6 and y = 2, using normalized query values - Expected GPA is: 3.33


For x = 6 and y = 2, using original query values - Expected GPA is: 3.33


#### Q 15. According to the (k = 5)-NN algorithm where Euclidean L2 distance is used on the normalized dataset in Q 13, what is the expected GPA if x = 6 and y = 2?

In [29]:
query_set = [6, 2]
normalized_query_set = [1, 0]

predicted_GPA= get_nearest_neighbors_prediction(train_set, normalized_query_set, k_neighbors=5, distancemethod ='euclidean')

print('\nFor x = 6 and y = 2, using normalized query values - Expected GPA is:' , predicted_GPA)

predicted_GPA1= get_nearest_neighbors_prediction(train_set, query_set, k_neighbors=5, distancemethod ='euclidean')

print('\n\nFor x = 6 and y = 2, using original query values - Expected GPA is:' , predicted_GPA1)




For x = 6 and y = 2, using normalized query values - Expected GPA is: 3


For x = 6 and y = 2, using original query values - Expected GPA is: 3


#### Q 16. According to the (p = 2) distance weighted (k = 5)-NN algorithm where Euclidean L2 distance is used on the normalized dataset in Q 13, what is the expected GPA if x = 6 and y = 2?

In [30]:
query_set = [6, 2]
normalized_query_set = [1, 0]

predicted_GPA= get_nearest_neighbors_prediction(train_set, normalized_query_set, k_neighbors=5, distancemethod ='euclidean', p=2)

print('\nFor x = 6 and y = 2, distance weighted Expected GPA is:' , predicted_GPA)



For x = 6 and y = 2, distance weighted Expected GPA is: 3.1


#### Q 17. According to the (k = 5)-NN algorithm where Manhattan L1 distance is used on the normalized dataset in Q 13, what is the expected GPA if x = 6 and y = 2?

In [31]:
query_set = [6, 2]
normalized_query_set = [1, 0]

predicted_GPA= get_nearest_neighbors_prediction(train_set, normalized_query_set, k_neighbors=5, distancemethod ='manhattan')

print('\nFor x = 6 and y = 2, Expected GPA is:' , predicted_GPA)



For x = 6 and y = 2, Expected GPA is: 2.2


### Functions to calculate residual for Q18-Q20

In [32]:
def predict_regression(train_set, test_row, k_neighbors, test_row_index, p):
    distances = list()
    neighbors = list()
     
    for train_row in train_set:
        train_row_index = train_set.index(train_row)
        
        #check if train index != test index
        if train_row_index != test_row_index:
            dist = euclidean_distance(test_row, train_row)
            distances.append((train_row, dist))
    
    #print('distances', distances)
    
    # sort the distances
    distances.sort(key=lambda tup: tup[1])
    
    # if p>0 i]that means we have to calculated weighted distances  
    if p > 0:
        # using first k distances
        kneighbor = distances[:k_neighbors]

        #call function to calculate weighted distances   
        prediction = get_distance_weighted_neighbour(kneighbor, p)
    
    else:
        for i in range(k_neighbors):
            neighbors.append(distances[i][0])
        
        output_values = [row[-1] for row in neighbors]         
        prediction = round(mean(output_values), 2)
    
    return prediction      
    

In [33]:
def compute_rsquare(test_set_predictions, no_of_attr, p, residual_type):
    sum = 0
    n = len(test_set_predictions)
     # Iterate all rows using DataFrame.iterrows()
    for index, row in test_set_predictions.iterrows():
        # avarage of (Actual - predicted)**2
        sum+= (row['predicted'] - row['g'])**2
        
        avg = sum/n
        #calculate sample variance of actual grades
        svar = variance(test_set_predictions['g'])
        
    if residual_type == 'rsquare':
        residual_val = 1 - (avg/svar)
    else:
        
        ssr = avg/(n-no_of_attr-1)
        ssd = svar/(n-1)
        residual_val = ssr/ssd
      
    return residual_val

In [34]:
# kNN Algorithm
# Parameters 
    # train is the train dataset
    # test is the test dataset
    # k_neighbors = no. of numbers need to find out
    # p = power for weighted distance
    # residual_type = rsquare or adjusted residual default is rsquare
    
def get_residual(train, test, k_neighbors, p = 0, residual_type = 'rsquare'):
    if 'predicted' in test.columns:
        test = test.drop(['predicted'], axis = 1)
    
    predictions = list()
    
    train_list = train.values.tolist()
    test_list = test.values.tolist()
    
    for row in test_list:
        test_row_index = test_list.index(row)
        output = predict_regression(train_list, row, k_neighbors,test_row_index, p)
        predictions.append(output)
    
    test_set_predictions = train.copy()
    test_set_predictions['predicted'] = predictions
    print('\nDataset with Predicted values\n')
    print(test_set_predictions)
    # print(type(test_set_predictions)
    
    no_of_attr = len(train.columns)-1
    
    rsquare_val = compute_rsquare(test_set_predictions, no_of_attr, p, residual_type)
    print('\n=============================================')
    print('Computed {0} R^2 value'.format(residual_type))
    print('=============================================')
    
    print(round(rsquare_val,2))

#### Q 18. Using the leave one out method, find the residual, R2 of the nearest neighbor algorithm

In [35]:
test_set1 = train_set.copy()

rSquare_residual = get_residual(train_set, test_set1, 1)



Dataset with Predicted values

   x  y  g  predicted
0  5  2  4          3
1  6  5  3          2
2  4  2  3          4
3  3  3  3          3
4  4  7  2          2
5  3  6  2          2
6  1  8  1          2

Computed rsquare R^2 value
0.4


#### Q 19. Using the leave one out method, find the residual, R2 of the (k = 3)-NN algorithm.

In [36]:

rSquare_residual = get_residual(train_set, test_set1, 3)



Dataset with Predicted values

   x  y  g  predicted
0  5  2  4       3.00
1  6  5  3       2.67
2  4  2  3       3.33
3  3  3  3       3.00
4  4  7  2       2.00
5  3  6  2       2.00
6  1  8  1       2.33

Computed rsquare R^2 value
0.55


#### Q 20. Using the leave one out method, find the adjusted R2, R¯2 of the (p = 2) distance weighted (k = 3)-NN algorithm.

In [37]:

rSquare_residual = get_residual(train_set, test_set1, 3, p=2, residual_type ='adjustedR2')



Dataset with Predicted values

   x  y  g  predicted
0  5  2  4   3.000000
1  6  5  3   2.615385
2  4  2  3   3.634146
3  3  3  3   3.109589
4  4  7  2   2.034483
5  3  6  2   1.981132
6  1  8  1   2.132890

Computed adjustedR2 R^2 value
0.64
